In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/workspaces/SebastianBot")

In [4]:
from cloud.functions.infrastructure.google.helper import load_google_credentials

credentials = load_google_credentials()

In [3]:
from datetime import datetime, timezone, timedelta

from sebastian.clients.google.gmail.query_builder import GmailQueryBuilder

time_threshold = datetime.now(timezone.utc) - timedelta(hours=720)

query = GmailQueryBuilder() \
        .from_email("order-update@amazon.de") \
        .subject("Ihr Paket kann bei DHL", exact=True) \
        .after_date(time_threshold) \
        .build()

In [4]:
from sebastian.clients.google.gmail.client import GmailClient


client = GmailClient(credentials=credentials)

In [5]:
mails = client.fetch_mails(query)
mails

[FullMailResponse(id='19a92064fd3105f7', threadId='19a92064fd3105f7', labelIds=['CATEGORY_UPDATES', 'INBOX'], snippet='Amazon.de Guten Tag Oliver Rüger, Ihr Paket mit 1 Artikel kann bei DHL Packstation 158 abgeholt werden. Sie erhalten den erforderlichen Abholcode in einer separaten Nachricht direkt vom Transporteur.', raw_payload={'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'oneironaut.oml@gmail.com'}, {'name': 'Received', 'value': 'by 2002:ac4:82cc:0:b0:759:93fa:ded6 with SMTP id 12csp3436196piv;        Mon, 17 Nov 2025 05:34:38 -0800 (PST)'}, {'name': 'X-Google-Smtp-Source', 'value': 'AGHT+IF7rxIMpc7FvDWnZSWtp6+lGvKuNi+vx9DZFUEenWXqD5XjsGZ2fqqndTpVpYSF9EfNTzfO'}, {'name': 'X-Received', 'value': 'by 2002:a05:6000:2dc7:b0:425:6fb5:2add with SMTP id ffacd0b85a97d-42b52814f34mr15028147f8f.19.1763386478356;        Mon, 17 Nov 2025 05:34:38 -0800 (PST)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1763386478; cv

In [6]:
from sebastian.usecases.DeliveryReady.parsing import parse_dhl_pickup_email_html


testi = mails[0]
parsed = parse_dhl_pickup_email_html(testi.payload)

In [7]:
parsed

PickupData(tracking_number='JJD000390019196495525', pickup_location='Packstation 158, Südhöhe 38', due_date='24.11.2025', preview='2-Pack Geschenkpapier...')

# Using the DeliveryReady Service

Now let's test the service that encapsulates the full workflow:

In [8]:
from sebastian.usecases.DeliveryReady.service import DeliveryReadyService

service = DeliveryReadyService(gmail_client=client)
result = service.get_recent_dhl_pickups(hours_back=72)

print(f"Found {len(result.item or [])} pickups")
if result.errors:
    print(f"Errors: {result.errors_string}")
    
result.item

Found 0 pickups


[]